In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym

In [3]:
lstm1 = nn.LSTM(1,64)
lstm2 = nn.LSTM(64,1)

criterion = nn.MSELoss()
optimizer = optim.Adam(chain(lstm1.parameters(),lstm2.parameters()), lr=0.0001)
for i in range(500):
    data = np.sin(np.linspace(0,5,100)+2*np.pi*np.random.rand())
    xs = data[:-1]
    ys = data[1:]
    X = Variable(torch.Tensor(xs).view(-1,1,1))
    y = Variable(torch.Tensor(ys))
    lstm1.zero_grad()
    lstm2.zero_grad()
    lstm_out, _ = lstm1(X,None)
    lstm_out, _ = lstm2(lstm_out,None)
    loss = criterion(lstm_out,y)
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print("i {}, loss {}".format(i,loss.data.numpy()[0]))

NameError: name 'chain' is not defined

In [ ]:
preds = []
hc1 = None
hc2 = None
# warm up
for i in range(len(xs)):
    x = Variable(torch.Tensor(1,1,1).fill_(xs[i]))
    tmp, hc1 = lstm1(x,hc1)
    x, hc2 = lstm2(tmp,hc2)
    preds.append(x.data.numpy()[0][0][0])
# x = Variable(torch.rand(1,1,1))
for i in range(500):
    tmp, hc1 = lstm1(x,hc1)
    x, hc2 = lstm2(tmp,hc2)
    preds.append(x.data.numpy()[0][0][0])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(preds)
# plt.plot(ys)

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,64)
        self.lstm2 = nn.LSTM(64,1)
        self.p = 0.5
        
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        X_in = torch.unsqueeze(seq[0],0)
        for X in seq.chunk(seq.size(0),dim=0):
            if np.random.rand()>self.p:
                X_in = X
            tmp, hc1 = self.lstm1(X_in,hc1)
            X_in, hc2 = self.lstm2(tmp,hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)


seq = Sequence()                
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.001)
for i in range(1000):
    data = np.sin(np.linspace(0,10,100)+2*np.pi*np.random.rand())
    xs = data[:-1]
    ys = data[1:]
    X = Variable(torch.Tensor(xs).view(-1,1,1))
    y = Variable(torch.Tensor(ys))
    if i%100==0:
        seq.p = min(seq.p+0.1,0.85)
    optimizer.zero_grad()
    lstm_out,_ = seq(X)
    loss = criterion(lstm_out[20:].view(-1),y[20:])
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print("i {}, loss {}".format(i,loss.data.numpy()[0]))            

In [ ]:
y = np.sin(np.linspace(0,100,1000)+2*np.pi*np.random.rand())
X = Variable(torch.Tensor(y))[:100].view(-1,1,1)
lstm_out,hc = seq(X)
preds = []
pred = lstm_out[-1].view(1,1,1)
for i in range(1000):
    pred,hc = seq(pred,hc)
    preds.append(pred.data.numpy().ravel()[0])

plt.figure(figsize=[10,8])
xs = np.arange(lstm_out.size(0))
plt.plot(xs,lstm_out.data.numpy().ravel())
xs = np.arange(1000)+len(xs)
plt.plot(xs,preds)
plt.plot(y)
plt.legend(['seeding','LSTM','real'])
plt.title('LSTM')

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.RNN(1,64,1)
        self.lstm2 = nn.RNN(64,1,1)
        self.p = 0.5
        
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        X_in = torch.unsqueeze(seq[0],0)
        for X in seq.chunk(seq.size(0),dim=0):
            if np.random.rand()>self.p:
                X_in = X
            tmp, hc1 = self.lstm1(X_in,hc1)
            X_in, hc2 = self.lstm2(tmp,hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)


seq = Sequence()                
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.001)
for i in range(1000):
    data = np.sin(np.linspace(0,10,100)+2*np.pi*np.random.rand())
    xs = data[:-1]
    ys = data[1:]
    X = Variable(torch.Tensor(xs).view(-1,1,1))
    y = Variable(torch.Tensor(ys))
    if i%100==0:
        seq.p = min(seq.p+0.1,0.8)
    optimizer.zero_grad()
    lstm_out,_ = seq(X)
    loss = criterion(lstm_out[20:].view(-1),y[20:])
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print("i {}, loss {}".format(i,loss.data.numpy()[0]))  

In [ ]:
y = np.sin(np.linspace(0,100,1000)+2*np.pi*np.random.rand())
X = Variable(torch.Tensor(y))[:100].view(-1,1,1)
lstm_out,hc = seq(X)
preds = []
pred = lstm_out[-1].view(1,1,1)
for i in range(1000):
    pred,hc = seq(pred,hc)
    preds.append(pred.data.numpy().ravel()[0])

plt.figure(figsize=[10,8])
xs = np.arange(lstm_out.size(0))
plt.plot(xs,lstm_out.data.numpy().ravel())
xs = np.arange(1000)+len(xs)
plt.plot(xs,preds)
plt.plot(y)
plt.legend(['seeding','RNN','real'])
plt.title('RNN')

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.GRU(1,64,1)
        self.lstm2 = nn.GRU(64,1,1)
        self.p = 0.5
        
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        X_in = torch.unsqueeze(seq[0],0)
        for X in seq.chunk(seq.size(0),dim=0):
            if np.random.rand()>self.p:
                X_in = X
            tmp, hc1 = self.lstm1(X_in,hc1)
            X_in, hc2 = self.lstm2(tmp,hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)


seq = Sequence()                
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.001)
for i in range(1000):
    data = np.sin(np.linspace(0,10,100)+2*np.pi*np.random.rand())
    xs = data[:-1]
    ys = data[1:]
    X = Variable(torch.Tensor(xs).view(-1,1,1))
    y = Variable(torch.Tensor(ys))
    if i%100==0:
        seq.p = min(seq.p+0.1,0.8)
    optimizer.zero_grad()
    lstm_out,_ = seq(X)
    loss = criterion(lstm_out[20:].view(-1),y[20:])
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print("i {}, loss {}".format(i,loss.data.numpy()[0]))  

In [ ]:
y = np.sin(np.linspace(0,100,1000)+2*np.pi*np.random.rand())
X = Variable(torch.Tensor(y))[:100].view(-1,1,1)
lstm_out,hc = seq(X)
preds = []
pred = lstm_out[-1].view(1,1,1)
for i in range(1000):
    pred,hc = seq(pred,hc)
    preds.append(pred.data.numpy().ravel()[0])

plt.figure(figsize=[10,8])
xs = np.arange(lstm_out.size(0))
plt.plot(xs,lstm_out.data.numpy().ravel())
xs = np.arange(1000)+len(xs)
plt.plot(xs,preds)
plt.plot(y)
plt.legend(['seeding','GRU','real'])
plt.title('GRU')